<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Supress cell output
%%capture
# Install knmy
!pip install knmy

In [0]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
%cd "/gdrive/My Drive/tensor-flow-state/tensor-flow-state"

/gdrive/My Drive/tensor-flow-state/tensor-flow-state


In [0]:
import pandas as pd
traffic = pd.read_csv("data/df_imputed_week_shift_added_holiday_weekends_speed_limit_130.csv", index_col = 0, parse_dates = True)

In [0]:
from knmy import knmy
import pandas as pd
import numpy as np
import datetime

def get_weather(traffic_df):
    freq = pd.infer_freq(traffic_df.index)
    start = (traffic.index[0] + datetime.timedelta(hours = 1)).strftime("%Y%m%d%H")
    end = (traffic.index[-1] + datetime.timedelta(hours = 1)).strftime("%Y%m%d%H") # this creates an extra day, but workaround would be tedious due to 00/24 difference
    *_, data = knmy.get_hourly_data(stations = [260], start = start, end = end, inseason = False, variables = ["TEMP", "PRCP", "SUNR"], parse = True)
    # Drop first row (it's a duplicate header)
    data.drop([0], inplace = True)
    # Switch from end of hour (KNMI convention) to start of hour (NDW convention)
    data.HH = (data.HH.astype(int) - 1).astype(str)
    # Remove the extra day at the end
    data = data[:-23]
    # Create datetime index
    data.index = pd.to_datetime(data.YYYYMMDD.astype(str) + data.HH.str.zfill(2), format = "%Y%m%d%H", yearfirst = True)
    # Select wanted columns
    data = data[["T", "TD", "DR", "RH", "SQ", "Q"]]
    # Rename columns
    data.rename(columns = {"T": "temp", "TD": "dew_point_temp", "DR": "precip_duration", "RH": "precip_sum", "SQ": "sun_duration", "Q": "sun_radiation"}, 
                inplace = True)
    # Switch to numeric dtypes
    data = data.astype(np.int32)
    # Switch to to Celsius and mm instead of dC and dmm
    data = data.div([10, 10, 1, 10, 1, 1], axis = 1)

    
    return data

In [0]:
weather = get_weather(traffic)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  # Remove the CWD from sys.path while we load stuff.


In [0]:
weather = weather.resample("T").pad()[:-1]

In [0]:
traffic.index.name = "time"
weather.index.name = "time"

In [0]:
df = traffic.merge(weather, how = 'left', on = "time")

In [0]:
df.to_csv("data/final_data.csv")

In [0]:
# T        = Temperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming; 
# TD       = Dauwpuntstemperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming;
# DR       = Duur van de neerslag (in 0.1 uur) per uurvak; 
# RH       = Uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm); 
# SQ       = Duur van de zonneschijn (in 0.1 uren) per uurvak, berekend uit globale straling  (-1 for <0.05 uur); 
# Q        = Globale straling (in J/cm2) per uurvak;